# RFP: Maze Solvers

## Project Overview
You are invited to submit a proposal that answers the following question:

### What path will your elf take?

*Please submit your proposal by **2/11/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, use [Gymnasium](https://gymnasium.farama.org/) to set up a [Frozen Lake maze](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) for your project. When you are done with the set up, describe the reward system you plan on using.

*Note, a level 5 maze is at least 10 x 10 cells large and contains at least five lake cells.*

In [1]:
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import random
import numpy as np

In [2]:
# Make maze

env = gym.make('FrozenLake-v1', desc=generate_random_map(size=10, seed=259),  is_slippery=False)
#initial_state = env.reset()

#env.render()

# Take a step (0: LEFT, 1: DOWN, 2: RIGHT, 3: UP)
#action = 2
#new_state, reward, terminated, truncated, info = env.step(action)

#env.render()

In [3]:
# Get state and action sizes
state_size = env.observation_space.n
action_size = env.action_space.n

# Extract hole ('H'), empty ('F'), and goal ('G') positions
desc = env.unwrapped.desc
hole_states = {i for i, row in enumerate(desc.flatten()) if row == b'H'}
empty_states = {i for i, row in enumerate(desc.flatten()) if row == b'F'}
goal_state = {i for i, row in enumerate(desc.flatten()) if row == b'G'}

# Initialize Q-table with small random values to encourage exploration
qtable = np.random.uniform(low=-0.5, high=0.5, size=(state_size, action_size))

# Training parameters
total_episodes = 20000  # Increase episodes for better learning
max_steps = 200  
learning_rate = 0.8  # Keep high at the start for faster learning
gamma = 0.95  # Higher discount factor to encourage long-term rewards
epsilon = 1.0  
min_epsilon = 0.01  # Allow more exploration for longer
decay_rate = 0.0001  # Slower decay to ensure sufficient exploration

# Training loop
rewards = []
for episode in range(total_episodes):
    state, _ = env.reset()
    total_rewards = 0
    done = False

    for step in range(max_steps):
        # Choose action using epsilon-greedy strategy
        if random.uniform(0, 1) > epsilon:
            action = np.argmax(qtable[state])  # Exploit
        else:
            action = env.action_space.sample()  # Explore

        # Take action
        new_state, reward, done, truncated, _ = env.step(action)

        # Modify rewards
        if new_state in hole_states:
            reward = -100.0  # Higher penalty for falling in a hole
        elif new_state in empty_states:
            reward = 1  # Small reward for moving forward
        elif new_state in goal_state:
            reward = 100.0  # Higher reward for reaching goal

        # Q-learning update
        qtable[state, action] = qtable[state, action] + learning_rate * (
            reward + gamma * np.max(qtable[new_state]) - qtable[state, action]
        )

        total_rewards += reward
        state = new_state

        if done or truncated:
            break

    # Decay epsilon slower for better exploration
    epsilon = max(min_epsilon, epsilon * np.exp(-decay_rate * episode))
    rewards.append(total_rewards)

# Print results
print("Score over time:", sum(rewards) / total_episodes)
print("Final Q-Table:")
print(qtable)

Score over time: 9.921334999999347
Final Q-Table:
[[ 1.90000000e+00  2.00000000e+00  2.00000000e+00  1.90000000e+00]
 [ 1.90000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00]
 [ 2.00000000e+00 -8.33255114e-01  1.99999994e+00  2.00000000e+00]
 [ 2.00000000e+00  1.91077672e+00 -6.73081636e-01  1.99845810e+00]
 [ 3.44111800e-01 -7.17785709e-03 -4.75577979e-02 -4.07164647e-01]
 [ 8.91339039e-03  3.24248953e-02 -3.13375948e-01 -4.24134246e-01]
 [ 4.36977272e-01 -2.06039508e-01 -6.63491848e-02 -4.84089428e-01]
 [-1.16674758e-01  1.11175983e-01  2.25949832e-01 -2.25880921e-01]
 [-5.13303297e-02 -4.28434611e-01 -4.44838857e-01 -4.28946033e-01]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00  1.90000000e+00]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00]
 [ 2.00000000e+00  2.00000000e+00 -8.33255114e-01  2.00000000e+00]
 [ 1.75520933e-01  5.30920176e-02 -3.76937193e-01  1.17477076e-01]
 [-8.3158649

In [ ]:
env.close()

#### Describe your reward system here.

### 2. Training Your Model
In the cell seen below, write the code you need to train a Q-Learning model. Display your final Q-table once you are done training your model.

*Note, level 5 work uses only the standard Python library and Pandas to train your Q-Learning model. A level 4 uses external libraries like Baseline3.*

In [ ]:
# Train model here.
# Don't forget to display your final Q table!

### 3. Testing Your Model
In the cell seen below, write the code you need to test your Q-Learning model for **1000 episodes**. It is important to test your model for 1000 episodes so that we are all able to compare our results.

*Note, level 5 testing uses both a success rate and an average steps taken metric to evaluate your model. Level 4 uses one or the other.*

In [ ]:
# Test model here.

### 4. Final Answer
In the first cell below, describe the path your elf takes to get to the gift. *Note, a level 5 answer includes a gif of the path your elf takes in order to reach the gift.*

In the second cell seen below, describe how well your Q-Learning model performed. Make sure that you explicitly name the **learning rate**, **the discount factor**, and the **reward system** that you used when training your final model. *Note, a level 5 description describes the model's performance using two types of quantitative evidence.*

![example image](https://gymnasium.farama.org/_images/frozen_lake.gif)

#### Describe the path your elf takes here.

#### Describe how well your Q-Learning model performed here.